<span style="font-size:36px;">Implement LogisticRegression</span>

In [348]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split

SEED = 1234
np.random.seed(SEED)


class LogisticRegression(object):
    def __init__(self, isNormalize) -> None:
        self.isNormalize = isNormalize

    def train(
        self,
        X_train,
        Y_train,
        weights,
        bias,
        learning_rate,
        iterations,
    ):
        if self.isNormalize:
            self.X_mean = np.mean(X_train, axis=0)
            self.X_std = np.std(X_train, axis=0)
            X_train = self.normalize(X_train)

        w_history, b_history, cost_history = self.gradient_descent(
            X_train, Y_train, weights, bias, learning_rate, iterations
        )

        w_best, b_best = w_history[-1], b_history[-1]
        train_history = (w_history, b_history, cost_history)

        return w_best, b_best, train_history

    def sigmoid(self, z):
        # Prevent overflow
        if not self.isNormalize:
            z = np.clip(z, -500, 1500)

        sigmoid_values = 1 / (1 + np.exp(-z))
        if not self.isNormalize:
            epsilon = 1e-10
            sigmoid_values = np.where(
                sigmoid_values == 1, sigmoid_values - epsilon, sigmoid_values
            )
            sigmoid_values = np.where(
                sigmoid_values == 0, sigmoid_values + epsilon, sigmoid_values
            )
        return sigmoid_values

    def model(self, X, w, b):
        logist = np.matmul(X, w) + b
        return self.sigmoid(logist)

    def normalize(self, X):
        return (X - self.X_mean) / self.X_std

    def cross_entropy(self, y_predict, y):
        return -np.mean(y * np.log(y_predict) + (1 - y) * np.log(1 - y_predict))

    def compute_cost(self, X, y, w, b):
        y_predict = self.model(X, w, b)
        return self.cross_entropy(y_predict, y)

    def compute_gradient(self, X, y, w, b):
        N = len(X)
        y_predict = self.model(X, w, b)
        delta = y_predict - y
        dw = 1 / N * np.matmul(X.T, delta)
        db = 1 / N * np.sum(delta)
        return dw, db

    def step_gradient(self, X, y, w, b, learning_rate):
        dw, db = self.compute_gradient(X, y, w, b)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        return w, b

    def gradient_descent(self, X, y, w_start, b_start, learning_rate, num_epochs):
        w, b = w_start, b_start

        cost_history = []
        w_history = []
        b_history = []

        for _ in range(num_epochs):
            cost_history.append(self.compute_cost(X, y, w, b))
            w, b = self.step_gradient(X, y, w, b, learning_rate)
            w_history.append(w)
            b_history.append(b)
        return w_history, b_history, cost_history

    def classify(self, y_predict):
        return np.where(y_predict > 0.5, 1, 0)

    # Evaluation
    def accuracy(self, y_predict, y):
        y_predict_class = np.where(y_predict > 0.5, 1, 0)
        return np.mean(y_predict_class == y)

    def sensitivity(self, y_predict, y):
        y_predict_class = self.classify(y_predict)
        TP = np.sum((y_predict_class == 1) & (y == 1))
        FN = np.sum((y_predict_class == 0) & (y == 1))
        if TP + FN == 0:
            return 0
        return TP / (TP + FN)

    def specificity(self, y_predict, y):
        y_predict_class = self.classify(y_predict)
        TN = np.sum((y_predict_class == 0) & (y == 0))
        FP = np.sum((y_predict_class == 1) & (y == 0))
        if TN + FP == 0:
            return 0
        return TN / (TN + FP)

    def precision(self, y_predict, y):
        y_predict_class = self.classify(y_predict)
        TP = np.sum((y_predict_class == 1) & (y == 1))
        FP = np.sum((y_predict_class == 1) & (y == 0))
        if TP + FP == 0:
            return 0
        return TP / (TP + FP)
    
    def ROC(self, y_predict, y):
        FPR = []
        TPR = []
        thresholds = np.sort(np.unique(y_predict))[::-1]

        for threshold in thresholds:
            binary_predictions = (y_predict >= threshold).astype(int)
            TPR.append(self.sensitivity(binary_predictions, y))
            FPR.append(1 - self.specificity(binary_predictions, y))

        ROC_points = FPR, TPR
        return ROC_points

    def AUROC(self, FPR, TPR):
        return np.trapz(TPR, FPR)
    
    def F1(self, y_predict, y):
        precision = self.precision(y_predict, y)
        sensitivity = self.sensitivity(y_predict, y)
        if precision + sensitivity == 0:
            return 0
        return 2 * (precision * sensitivity) / (precision + sensitivity)

<span style="font-size:36px;">Function to show results for specific parameters</span>

In [349]:
def show_results(
    X_train,
    y_train,
    X_test,
    y_test,
    weights,
    bias,
    learning_rate,
    iterations,
    isNormalize,
):
    algorithm = LogisticRegression(isNormalize)

    w_best, b_best, train_history = algorithm.train(
        X_train, y_train, weights, bias, learning_rate, iterations
    )

    # TRAIN RESULTS

    # Cost function
    w_history, b_history, cost_history = train_history
    fig_cost = px.line(
        x=range(len(cost_history)),
        y=cost_history,
        title="Cost Function",
        labels={"x": "Epoch", "y": "Cost"},
    )

    if isNormalize:
        X_train = algorithm.normalize(X_train)
    y_predict = algorithm.model(X_train, w_best, b_best)
    y_class = algorithm.classify(y_predict)
    accuracy = algorithm.accuracy(y_predict, y_train)
    F1 = algorithm.F1(y_predict, y_train)

    # Sample predictions
    y_train_sample = y_train[:5]
    y_predict_sample = y_predict[:5]
    y_class_sample = y_class[:5]

    df_train_sample = pd.DataFrame(
        {
            "y_train": y_train_sample.flatten(),
            "y_class": y_class_sample.flatten(),
            "y_predict": y_predict_sample.flatten(),
        }
    )

    # ROC Curve
    ROC_points = algorithm.ROC(y_predict, y_train)
    FPR, TPR = ROC_points

    df_train_ROC = pd.DataFrame({"FPR": FPR, "TPR": TPR})
    fig_train_ROC = px.line(
        df_train_ROC,
        x="FPR",
        y="TPR",
        markers=True,
        title="ROC Curve - Train",
        labels={"FPR": "False Positive Rate (FPR)", "TPR": "True Positive Rate (TPR)"},
    )

    # Print the train evaluations and sample predictions
    print("TRAIN")
    print(f"Train Accuracy: {accuracy}")
    print(f"Train F1: {F1}")
    print(f"Train AUROC: {algorithm.AUROC(FPR, TPR)}")
    print("\nSample predictions")
    print(df_train_sample)
    fig_cost.show()
    fig_train_ROC.show()

    # TEST RESULTS

    if isNormalize:
        X_test = algorithm.normalize(X_test)
    y_predict = algorithm.model(X_test, w_best, b_best)
    y_class = algorithm.classify(y_predict)
    accuracy = algorithm.accuracy(y_predict, y_test)
    F1 = algorithm.F1(y_predict, y_test)

    # Sample predictions
    y_test_sample = y_test[:5]
    y_predict_sample = y_predict[:5]
    y_class_sample = y_class[:5]

    df_test_sample = pd.DataFrame(
        {
            "y_test": y_test_sample.flatten(),
            "y_class": y_class_sample.flatten(),
            "y_predict": y_predict_sample.flatten(),
        }
    )

    # ROC curve
    ROC_points = algorithm.ROC(y_predict, y_test)
    FPR, TPR = ROC_points

    df_test_ROC = pd.DataFrame({"FPR": FPR, "TPR": TPR})
    fig_test_ROC = px.line(
        df_test_ROC,
        x="FPR",
        y="TPR",
        markers=True,
        title="ROC Curve - Test",
        labels={"FPR": "False Positive Rate (FPR)", "TPR": "True Positive Rate (TPR)"},
    )

    # Print the test evaluations and sample predictions
    print("TEST")
    print(f"Test Accuracy: {accuracy}")
    print(f"Test F1: {F1}")
    print(f"Test AUROC: {algorithm.AUROC(FPR, TPR)}")
    print("\nSample predictions")
    print(df_test_sample)
    fig_test_ROC.show()

<span style="font-size:36px;">Implement data and parameters</span>

In [350]:
df = pd.read_csv("data.csv")

X = df.drop(columns=["Diagnosis"]).values
y = df["Diagnosis"].values
N, D = X.shape
y = y.reshape(N, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=SEED
)

weights = np.random.randn(D, 1)
bias = np.random.randn()

<span style="font-size:36px;">Use LogisticRegression without Normalize</span>

In [351]:
show_results(X_train, y_train, X_test, y_test, weights, bias, 0.001, 100, False)

TRAIN
Train Accuracy: 0.7276995305164319
Train F1: 0.41414141414141414
Train AUROC: 0.7024838396514573

Sample predictions
   y_train  y_class      y_predict
0        0        0  6.418674e-144
1        1        0   1.060849e-30
2        0        0   5.546937e-60
3        1        0  1.721463e-155
4        0        0   2.693460e-77


TEST
Test Accuracy: 0.7272727272727273
Test F1: 0.45070422535211263
Test AUROC: 0.6386363636363637

Sample predictions
   y_test  y_class     y_predict
0       0        0  4.886255e-50
1       0        0  7.208140e-62
2       0        0  4.904707e-82
3       0        0  3.892975e-60
4       0        0  6.339042e-69


<span style="font-size:36px;">Use LogisticRegression with Normalize</span>

In [352]:
show_results(X_train, y_train, X_test, y_test, weights, bias, 0.1, 100, True)

TRAIN
Train Accuracy: 0.9553990610328639
Train F1: 0.939297124600639
Train AUROC: 0.9914521819430303

Sample predictions
   y_train  y_class  y_predict
0        0        0   0.122148
1        1        1   0.837923
2        0        0   0.007097
3        1        1   0.853440
4        0        0   0.123485


TEST
Test Accuracy: 0.951048951048951
Test F1: 0.9320388349514563
Test AUROC: 0.974586776859504

Sample predictions
   y_test  y_class  y_predict
0       0        0   0.029687
1       0        0   0.000375
2       0        0   0.283708
3       0        0   0.002135
4       0        0   0.011731


<span style="font-size:36px;">Data - select only some features</span>

In [353]:
df = pd.read_csv("data.csv")

X = df.iloc[:, 15:22].values
y = df["Diagnosis"].values
N, D = X.shape
X
y = y.reshape(N, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=SEED
)

weights = np.random.randn(D, 1)
bias = np.random.randn()

<span style="font-size:36px;">Use LogisticRegression without Normalize</span>

In [354]:
show_results(X_train, y_train, X_test, y_test, weights, bias, 0.001, 100, False)

TRAIN
Train Accuracy: 0.1431924882629108
Train F1: 0.11192214111922141
Train AUROC: 0.058887599744275806

Sample predictions
   y_train  y_class  y_predict
0        0        0   0.479481
1        1        1   0.502305
2        0        1   0.517525
3        1        0   0.494713
4        0        1   0.510298


TEST
Test Accuracy: 0.11888111888111888
Test F1: 0.08695652173913043
Test AUROC: 0.04008264462809917

Sample predictions
   y_test  y_class  y_predict
0       0        1   0.529731
1       0        1   0.551136
2       0        1   0.567124
3       0        1   0.572398
4       0        1   0.500792


<span style="font-size:36px;">Use LogisticRegression with Normalize</span>

In [355]:
show_results(X_train, y_train, X_test, y_test, weights, bias, 0.1, 100, True)

TRAIN
Train Accuracy: 0.8943661971830986
Train F1: 0.8543689320388349
Train AUROC: 0.9626595316458694

Sample predictions
   y_train  y_class  y_predict
0        0        0   0.395579
1        1        0   0.385385
2        0        0   0.210146
3        1        1   0.660972
4        0        0   0.278397


TEST
Test Accuracy: 0.8601398601398601
Test F1: 0.803921568627451
Test AUROC: 0.9508264462809917

Sample predictions
   y_test  y_class  y_predict
0       0        0   0.146874
1       0        0   0.078395
2       0        1   0.769120
3       0        0   0.045941
4       0        0   0.261297
